In [31]:
import pandas as pd

# Load data from CSV file
data = pd.read_csv('wireless_traffic_dataset.csv')

# Display the first few rows of the dataframe
data.head()

,Tower_ID,Signal_Strength(dBm),Network_Traffic(MB),Latency(ms),User_Count,Device_Type
0,Tower_8,-66,430.07,770,232,IoT Device
1,Tower_80,-79,495.16,485,337,Laptop
2,Tower_72,-63,269.30,20,326,IoT Device
3,Tower_55,-84,12.13,503,467,Tablet
4,Tower_69,-44,93.60,575,15,Smartphone


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Prepare the data
X = data[['Signal_Strength(dBm)', 'Network_Traffic(MB)', 'User_Count', 'Device_Type', ]]
y = data['Latency(ms)']

# One-hot encode the 'Device_Type' column
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['Device_Type'])
    ],
    remainder='passthrough'
)

# Create a pipeline that first transforms the data and then fits the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Calculate the mean absolute error
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R^2 score
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'R^2 Score: {r2}')

Mean Absolute Error: 252.15640757594335
R^2 Score: -0.015870916890303732


In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Sample dataset
# data = {
#     'SignalStrength': np.random.rand(100),
#     'NetworkTraffic': np.random.rand(100),
#     'UserCount': np.random.randint(1, 100, size=100),
#     'DeviceType': np.random.choice(['Mobile', 'Desktop'], size=100),
#     'NetworkLatency': np.random.rand(100)
# }

# df = pd.DataFrame(data)

# One-hot encode categorical features
data = pd.get_dummies(data, columns=['Device_Type', 'Tower_ID'])

# Split dataset into network-related and user behavior-related datasets
X_network = data[['Signal_Strength(dBm)', 'Network_Traffic(MB)']]
X_user_behavior = data.drop(columns=['Latency(ms)', 'Signal_Strength(dBm)', 'Network_Traffic(MB)'])
y = data['Latency(ms)']

# Standardize the features if necessary (optional)
scaler_X_network = StandardScaler()
X_network = scaler_X_network.fit_transform(X_network)

scaler_X_user_behavior = StandardScaler()
X_user_behavior = scaler_X_user_behavior.fit_transform(X_user_behavior)

# Split data into training and testing sets
X_network_train, X_network_test, X_user_behavior_train, X_user_behavior_test, y_train, y_test = train_test_split(
    X_network, X_user_behavior, y, test_size=0.2, random_state=42)

# Train Model A (Network-related features)
model_A = LinearRegression()
model_A.fit(X_network_train, y_train)

# Train Model B (User behavior-related features)
model_B = LinearRegression()
model_B.fit(X_user_behavior_train, y_train)

# Generate predictions from both models
predictions_A = model_A.predict(X_network_test)
predictions_B = model_B.predict(X_user_behavior_test)

# Combine outputs of Model A and Model B to form a new dataset for the meta-model
meta_features = np.vstack((predictions_A, predictions_B)).T

# Split combined features into training and testing sets for the meta-model
meta_train_features, meta_test_features, y_train_meta, y_test_meta = train_test_split(
    meta_features, y_test, test_size=0.2, random_state=42)

# Train Meta-Model
meta_model = LinearRegression()
meta_model.fit(meta_train_features, y_train_meta)

# Predict and evaluate the meta-model
final_predictions = meta_model.predict(meta_test_features)
mse = mean_squared_error(y_test_meta, final_predictions)
print(f'Mean Squared Error of the combined model: {mse:.4f}')
mae = mean_absolute_error(y_test_meta, final_predictions)
print(f'Mean Absolute Error of the combined model: {mae:.4f}')
# Calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')

Mean Squared Error of the combined model: 84976.2326
Mean Absolute Error of the combined model: 257.0556
Mean Absolute Error: 257.0556360945673
R^2 Score: -0.015870916890303732
